In this example a short constraint free NVT simulation of the molecule perylene under vacuum conditions will be used to generate shake constraints. By that the size of the simulation step can be increased up to 2 fs.

First, the PQAnalysis package is imported and the currently loaded version will be checked.

In [ ]:
import PQAnalysis
PQAnalysis._version.version

Then we read in the trajectory from a `.xyz` file generated by a constraint free `PQ` simulation.

Note: In this example notebook a trajectory consisting of just 500 frames is used due to convenience. For determining shake constraints a larger number of frames is highly recommended.

In [ ]:
from PQAnalysis.io import TrajectoryReader

input_file = "./perylene-md-01.xyz"
reader = TrajectoryReader(input_file)

traj = reader.read()
traj

For reasons of symmetry the four H-C bonds closer to the molecule center (green) should attain the same length. Equivalently, the outer eight H-C bonds (blue) should also be of equal length. According to this premise, hydrogen atoms are selected from the read trajectory and split into two groups.

In [ ]:
from PQAnalysis.topology import Selection
import numpy as np

selection = Selection("H")
h_indices = selection.select(traj.topology)

inner_hydrogen = np.array(h_indices)[[0, 5, 9, 10]]
outer_hydrogen = np.setdiff1d(h_indices, inner_hydrogen)

Lastly, the shake constraints are calculated with averaged bond distances for both H-C groups and printed to an output file `shake.top`. This file can directly be used as input for subsequent simulations.

Note: To enable constraint dynamics the shake keyword needs to be turned on as described in the documentation.

In [ ]:
from PQAnalysis.topology.shake_topology import ShakeTopologyGenerator

shake_generator = ShakeTopologyGenerator("H")
shake_generator.generate_topology(traj)
shake_generator.average_equivalents([inner_hydrogen, outer_hydrogen], comments=["inner", "outer"])
shake_generator.write_topology("shake.top", mode="w")

